# Model Management - Despliegue de modelos de Machine Learning

In [68]:
# %pip install mlflow
! source .env
! echo $MLFLOW_TRACKING_USERNAME

abdala9512


In [73]:
import mlflow
import dotenv
import json

from typing import List


In [99]:
config = dotenv.dotenv_values(".env")

os.environ["MLFLOW_TRACKING_USERNAME"] = config.get("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = config.get("MLFLOW_TRACKING_PASSWORD")

MLFLOW_TRACKING_URI = config.get("MLFLOW_TRACKING_URI")
MLFLOW_DSRP_EXPERIMENT = "DSRP Curso Despliegue de modelos de Machine Learning"
MLFLOW_DSRP_MODEL_NAME = "default_prediction_dsrp_model"

mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI,)
mlflow.set_experiment(experiment_name=MLFLOW_DSRP_EXPERIMENT)

<Experiment: artifact_location='mlflow-artifacts:/7452221b1c9343fa95e5588a86ea9b93', creation_time=1711567929054, experiment_id='0', last_update_time=1711567929054, lifecycle_stage='active', name='DSRP Curso Despliegue de modelos de Machine Learning', tags={}>

In [94]:
def search_best_model(
        experiment_names: List[str] = [],
        metric_name: str = "r2"
    ) -> str:
    """Search Best Run ID of given experiments
    """
    runs_  = mlflow.search_runs(experiment_names=experiment_names)
    best_run = runs_.loc[runs_[f'metrics.{metric_name}'].idxmax()]

    
    return best_run['run_id'], best_run["artifact_uri"]

def get_artifact_uri_production(model_name: str) -> str:

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        model = dict(mv)
        if model["current_stage"] == "Production":
            production_model = model

    _run_id = production_model.get("run_id")
    return mlflow.get_run(_run_id).info.artifact_uri

In [96]:
best_run_id, best_run_art_uri= search_best_model(experiment_names=["DSRP Curso Despliegue de modelos de Machine Learning"])

In [26]:
config.get("MLFLOW_TRACKING_URI")

'https://dagshub.com/abdala9512/dsrp-mlops-deployment.mlflow'

In [78]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/7452221b1c9343fa95e5588a86ea9b93', creation_time=1711567929054, experiment_id='0', last_update_time=1711567929054, lifecycle_stage='active', name='DSRP Curso Despliegue de modelos de Machine Learning', tags={}>]

In [84]:
runs_ = mlflow.search_runs(experiment_ids=["0"])
runs_.loc[runs_[f'metrics.r2'].idxmax()]['run_id']

'ffbc160e02d5463db20fd3fbda614673'

In [40]:
MLFLOW_TRACKING_URI

'https://dagshub.com/abdala9512/dsrp-mlops-deployment.mlflow'

In [100]:
mlflow.register_model(
    model_uri=f"runs:/{best_run_id}/{best_run_art_uri}",
    name=MLFLOW_DSRP_MODEL_NAME
)

Successfully registered model 'default_prediction_dsrp_model'.
2024/03/27 15:43:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: default_prediction_dsrp_model, version 1
Created version '1' of model 'default_prediction_dsrp_model'.


<ModelVersion: aliases=[], creation_timestamp=1711572197999, current_stage='None', description='', last_updated_timestamp=1711572197999, name='default_prediction_dsrp_model', run_id='ffbc160e02d5463db20fd3fbda614673', run_link='', source='mlflow-artifacts:/7452221b1c9343fa95e5588a86ea9b93/ffbc160e02d5463db20fd3fbda614673/artifacts/mlflow-artifacts:/7452221b1c9343fa95e5588a86ea9b93/ffbc160e02d5463db20fd3fbda614673/artifacts', status='READY', status_message='', tags={}, user_id='', version='1'>